In [1]:
import re
from typing import List, Optional
from langchain_core.documents import Document

def load_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def extract_chapter_meta(chapter_num: str, chapter_title: str) -> dict:
    """
    chapter_num이 '01' → level 1, parent 없음
    chapter_num이 '01-02' → level 2, parent '01'
    """
    if "-" in chapter_num:
        parent = chapter_num.split("-")[0]
        level = 2
    else:
        parent = None
        level = 1
    return {
        "chapter": chapter_num,
        "title": chapter_title,
        "level": level,
        "parent": parent
    }

def split_by_chapter(text: str) -> List[Document]:
    """
    Wikidocs 형식 텍스트를 챕터별로 나누고 계층 정보를 포함한 Document로 변환
    """
    pattern = r"(?=^---\s+(\d{2}(?:-\d{2})?)\.\s+(.*?)\s+---)"
    matches = list(re.finditer(pattern, text, flags=re.MULTILINE))

    documents = []
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        chapter_num = match.group(1)
        chapter_title = match.group(2).strip()
        chapter_text = text[start:end].strip()

        metadata = extract_chapter_meta(chapter_num, chapter_title)

        doc = Document(page_content=chapter_text, metadata=metadata)
        documents.append(doc)

    return documents

def load_and_split_wikidocs(path: str) -> List[Document]:
    text = load_txt(path)
    return split_by_chapter(text)

def filter_chapters(
    documents: List[Document],
    level: Optional[int] = None,
    parent: Optional[str] = None,
    contains_title: Optional[str] = None
) -> List[Document]:
    """
    챕터 리스트에서 조건에 맞는 챕터만 필터링
    - level: 1 (대챕터), 2 (소챕터)
    - parent: '01' 등 상위 챕터 번호
    - contains_title: 제목 키워드 포함 여부
    """
    filtered = []
    for doc in documents:
        meta = doc.metadata
        if level and meta["level"] != level:
            continue
        if parent and meta["parent"] != parent:
            continue
        if contains_title and contains_title not in meta["title"]:
            continue
        filtered.append(doc)
    return filtered

In [2]:
first_docs = load_and_split_wikidocs("C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt")

# 소챕터만 추출 (level 2)
subchapters = filter_chapters(first_docs, level=2)

# 대챕터 02의 모든 소챕터 추출
chap02_children = filter_chapters(first_docs, parent="02")

# "설치"라는 단어를 포함하는 챕터만 추출
install_sections = filter_chapters(first_docs, contains_title="설치")

print(f"📌 총 문서 수: {len(first_docs)}")
print(f"🔹 소챕터 수: {len(subchapters)}")
print(f"🔹 02번 챕터 하위 수: {len(chap02_children)}")
print(f"🔹 '설치' 포함 제목 수: {len(install_sections)}")

📌 총 문서 수: 165
🔹 소챕터 수: 0
🔹 02번 챕터 하위 수: 0
🔹 '설치' 포함 제목 수: 1


In [3]:
second_docs = load_and_split_wikidocs("C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_02.txt")

# 소챕터만 추출 (level 2)
subchapters = filter_chapters(second_docs, level=2)

# 대챕터 02의 모든 소챕터 추출
chap02_children = filter_chapters(second_docs, parent="02")

# "설치"라는 단어를 포함하는 챕터만 추출
install_sections = filter_chapters(second_docs, contains_title="설치")

print(f"📌 총 문서 수: {len(second_docs)}")
print(f"🔹 소챕터 수: {len(subchapters)}")
print(f"🔹 02번 챕터 하위 수: {len(chap02_children)}")
print(f"🔹 '설치' 포함 제목 수: {len(install_sections)}")

📌 총 문서 수: 27
🔹 소챕터 수: 0
🔹 02번 챕터 하위 수: 0
🔹 '설치' 포함 제목 수: 0


In [4]:
third_docs = load_and_split_wikidocs("C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_03.txt")

# 소챕터만 추출 (level 2)
subchapters = filter_chapters(third_docs, level=2)

# 대챕터 02의 모든 소챕터 추출
chap02_children = filter_chapters(third_docs, parent="02")

# "설치"라는 단어를 포함하는 챕터만 추출
install_sections = filter_chapters(third_docs, contains_title="설치")

print(f"📌 총 문서 수: {len(third_docs)}")
print(f"🔹 소챕터 수: {len(subchapters)}")
print(f"🔹 02번 챕터 하위 수: {len(chap02_children)}")
print(f"🔹 '설치' 포함 제목 수: {len(install_sections)}")

📌 총 문서 수: 33
🔹 소챕터 수: 4
🔹 02번 챕터 하위 수: 0
🔹 '설치' 포함 제목 수: 0


In [5]:
print(len(first_docs))
print(len(second_docs))
print(len(third_docs))

165
27
33


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 초기화 (하나의 스플리터만 사용)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 청크 크기
    chunk_overlap=50,  # 청크 간 중복
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # 분할 기준
)

first_splits = []
second_splits = []
third_splits = []

for doc in first_docs:
    splits = splitter.split_text(doc.page_content)
    first_splits.extend(splits)
    # print(first_splits)

for doc in second_docs:
    splits = splitter.split_text(doc.page_content)
    second_splits.extend(splits)
    # print(second_splits)

for doc in third_docs:
    splits = splitter.split_text(doc.page_content)
    third_splits.extend(splits)
    # print(third_splits)

print(f"첫 번째 문서 청크 수: {len(first_splits)}")
print(f"두 번째 문서 청크 수: {len(second_splits)}")
print(f"세 번째 문서 청크 수: {len(third_splits)}")

첫 번째 문서 청크 수: 4746
두 번째 문서 청크 수: 1983
세 번째 문서 청크 수: 2964


In [7]:
from sentence_transformers import SentenceTransformer
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
import shutil

# OpenAI 임베딩 모델 초기화
from dotenv import load_dotenv

# 환경 변수에서 API 키 로드
load_dotenv()

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 지정

# DB 디렉토리 존재 여부 확인 및 삭제
if os.path.exists(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\first_db"):
    shutil.rmtree(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\first_db")
if os.path.exists(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\second_db"):
    shutil.rmtree(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\second_db")
if os.path.exists(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\third_db"):
    shutil.rmtree(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\third_db")

# 첫 번째 문서 DB 생성
first_db = Chroma.from_texts(
    texts=first_splits,
    embedding=embeddings,
    persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\first_db"
)
first_db.persist()

# # 두 번째 문서 DB 생성
# second_db = Chroma.from_texts(
#     texts=second_splits,
#     embedding=embeddings,
#     persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\second_db"
# )
# second_db.persist()

# # 세 번째 문서 DB 생성
# third_db = Chroma.from_texts(
#     texts=third_splits,
#     embedding=embeddings,
#     persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\third_db"
# )
# third_db.persist()

C:\Users\user\AppData\Local\Temp\ipykernel_4640\1310670779.py:30: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  first_db.persist()


In [8]:
first_db.as_retriever(search_type="similarity", search_kwargs={"k": 10}).invoke('import pandas')


[Document(metadata={}, page_content='# 원하는 Pandas DataFrame을 정의합니다.\ndf = pd.read_csv("./data/titanic.csv")\ndf.head()'),
 Document(metadata={}, page_content='PassengerId\nSurvived\nPclass\nName\nSex\nAge\nSibSp\nParch\nTicket\nFare\nCabin\nEmbarked\n1\n0\n3\nBraund, Mr. Owen Harris\nmale\n22\n1\n0\nA/5 21171\n7.25\nS\n2\n1\n1\nCumings, Mrs. John Bradley (Florence Briggs Thayer)\nfemale\n38\n1\n0\nPC 17599\n71.2833\nC85\nC\n3\n1\n3\nHeikkinen, Miss. Laina\nfemale\nDataFrameLoader\nPandas는 Python 프로그래밍 언어를 위한 오픈 소스 데이터 분석 및 조작 도구입니다. 이 라이브러리는 데이터 과학, 머신러닝, 그리고 다양한 분야의 데이터 작업에 널리 사용되고 있습니다.\nimport pandas as pd\n# CSV 파일 읽기\ndf = pd.read_csv("./data/titanic.csv")\n첫 5개 행을 조회합니다.'),
 Document(metadata={}, page_content="연관키워드: 딥러닝, 자연어 처리, 시퀀스 모델링\n판다스 (Pandas)\nMetadata: {'source': './data/appendix-keywords.txt', 'id': 10, 'relevance_score': 0.9997084}"),
 Document(metadata={}, page_content='# !pip install -qU langchain-teddynote\nfrom langchain_teddynote import logging\n# 프로젝트 이름을 입력합니다.

In [9]:
# import re
# from typing import List
# from langchain_core.documents import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# def load_txt(path: str) -> str:
#     """
#     텍스트 파일을 UTF-8로 로드
#     """
#     with open(path, "r", encoding="utf-8") as f:
#         return f.read()

# def extract_chapter_meta(chapter_num: str, chapter_title: str) -> dict:
#     """
#     개선된 버전: 제목 내 [카테고리] → parent
#     """
#     # [카테고리]가 존재하면 parent로 추정
#     category_match = re.match(r"\[(.*?)\]", chapter_title)
#     if category_match:
#         parent = category_match.group(1)
#     else:
#         parent = None

#     return {
#         "chapter": chapter_num,
#         "title": chapter_title,
#         "level": 1,  # 실제 파일에는 대챕터만 존재
#         "parent": parent
#     }

# def protect_code_blocks(text: str) -> str:
#     """
#     ```로 감싸진 코드 블록을 <CODE_BLOCK>으로 감싸 보존
#     """
#     code_pattern = re.compile(r"```.*?\n.*?```", re.DOTALL)
#     protected = []
#     last_end = 0

#     for match in code_pattern.finditer(text):
#         start, end = match.span()
#         protected.append(text[last_end:start])
#         code = match.group()
#         protected.append(f"\n<CODE_BLOCK>\n{code}\n</CODE_BLOCK>\n")
#         last_end = end

#     protected.append(text[last_end:])
#     return "".join(protected)

# def split_by_chapter_with_code(text: str) -> List[Document]:
#     """
#     챕터 헤더(--- 01. 제목 ---) 기준으로 분할하되, 각 챕터 내 코드블록 보존
#     """
#     pattern = r"(?=^---\s+(\d{2}(?:-\d{2})?)\.\s+(.*?)\s+---)"
#     matches = list(re.finditer(pattern, text, flags=re.MULTILINE))

#     documents = []
#     for i, match in enumerate(matches):
#         start = match.start()
#         end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

#         chapter_num = match.group(1)
#         chapter_title = match.group(2).strip()
#         chapter_text = text[start:end].strip()

#         protected_text = protect_code_blocks(chapter_text)
#         metadata = extract_chapter_meta(chapter_num, chapter_title)

#         documents.append(Document(page_content=protected_text, metadata=metadata))

#     return documents

# def process_wikidocs_files(paths: List[str]) -> List[Document]:
#     """
#     여러 Wikidocs 형식의 텍스트 파일을 처리하여 Document 리스트로 반환
#     """
#     all_docs = []
#     for path in paths:
#         text = load_txt(path)
#         docs = split_by_chapter_with_code(text)
#         all_docs.extend(docs)
#     return all_docs

# def extract_and_replace_code_blocks(text: str):
#     """
#     <CODE_BLOCK>...</CODE_BLOCK> 구간을 [[CODE:0]], [[CODE:1]], ...로 치환
#     """
#     code_blocks = []
#     pattern = re.compile(r"<CODE_BLOCK>\s*```.*?\n.*?```\s*</CODE_BLOCK>", re.DOTALL)

#     def replacer(match):
#         code_blocks.append(match.group())
#         return f"[[CODE:{len(code_blocks) - 1}]]"

#     modified_text = pattern.sub(replacer, text)
#     return modified_text, code_blocks

# def split_protected_chunks(docs: List[Document], chunk_size=1000, chunk_overlap=200) -> List[Document]:
#     """
#     코드 블록이 잘리지 않도록 보호한 상태로 chunk 분할
#     """
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap
#     )

#     chunked_docs = []
#     for doc in docs:
#         # 코드 블록을 임시 토큰으로 치환
#         mod_text, code_blocks = extract_and_replace_code_blocks(doc.page_content)
#         temp_doc = Document(page_content=mod_text, metadata=doc.metadata)

#         # 분할
#         split_docs = splitter.split_documents([temp_doc])

#         # 코드 블록 복원
#         for d in split_docs:
#             restored_text = d.page_content
#             for i, block in enumerate(code_blocks):
#                 restored_text = restored_text.replace(f"[[CODE:{i}]]", block)
#             d.page_content = restored_text
#             chunked_docs.append(d)

#     return chunked_docs

In [52]:
import re
from typing import List
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def protect_code_blocks(text: str) -> str:
    code_pattern = re.compile(r"```.*?\n.*?```", re.DOTALL)
    protected = []
    last_end = 0
    for match in code_pattern.finditer(text):
        start, end = match.span()
        protected.append(text[last_end:start])
        code = match.group()
        protected.append(f"\n<CODE_BLOCK>\n{code}\n</CODE_BLOCK>\n")
        last_end = end
    protected.append(text[last_end:])
    return "".join(protected)

def split_by_structure_with_code(text: str) -> List[Document]:
    lines = text.splitlines()
    documents = []
    buffer = []

    current_chapter_num = None
    current_chapter_title = None
    current_section_num = None
    current_section_title = None

    chapter_pattern = re.compile(r"^---\s+CH(\d+)\s+(.*?)\s+---$")
    section_pattern = re.compile(r"^---\s+(\d{2})\.\s+(.*?)\s+---$")
    title_only_pattern = re.compile(r"^(---|===).*?(---|===)$")

    for line in lines:
        chapter_match = chapter_pattern.match(line)
        section_match = section_pattern.match(line)

        if chapter_match:
            # flush 이전 buffer
            if buffer:
                chunk = "\n".join(buffer).strip()
                # 제목만 있는 문서인지 확인
                if not is_title_only_document(chunk):
                    protected = protect_code_blocks(chunk)
                    documents.append(Document(
                        page_content=protected,
                        metadata={
                            "chapter_info": f"CH{current_chapter_num} {current_chapter_title}" if current_chapter_num else None,
                            "section_info": f"{current_section_num}. {current_section_title}" if current_section_num else None,
                        }
                    ))
                buffer = []

            current_chapter_num = chapter_match.group(1)
            current_chapter_title = chapter_match.group(2).strip()
            current_section_num, current_section_title = None, None

        elif section_match:
            if buffer:
                chunk = "\n".join(buffer).strip()
                # 제목만 있는 문서인지 확인
                if not is_title_only_document(chunk):
                    protected = protect_code_blocks(chunk)
                    documents.append(Document(
                        page_content=protected,
                        metadata={
                            "chapter_info": f"CH{current_chapter_num} {current_chapter_title}" if current_chapter_num else None,
                            "section_info": f"{current_section_num}. {current_section_title}" if current_section_num else None,
                        }
                    ))
                buffer = []

            current_section_num = section_match.group(1)
            current_section_title = section_match.group(2).strip()

        buffer.append(line)

    if buffer:
        chunk = "\n".join(buffer).strip()
        # 제목만 있는 문서인지 확인
        if not is_title_only_document(chunk):
            protected = protect_code_blocks(chunk)
            documents.append(Document(
                page_content=protected,
                metadata={
                    "chapter_info": f"CH{current_chapter_num} {current_chapter_title}" if current_chapter_num else None,
                    "section_info": f"{current_section_num}. {current_section_title}" if current_section_num else None,
                }
            ))

    return documents

def is_title_only_document(text: str) -> bool:
    """
    문서가 제목만 포함하고 있는지 확인합니다.
    """
    # 공백과 줄바꿈을 제거한 텍스트
    cleaned_text = text.strip()
    
    # 제목 패턴 (---로 시작하고 ---로 끝나거나, ===로 시작하고 ===로 끝나는 패턴)
    title_pattern = re.compile(r"^(---|===).*?(---|===)$", re.DOTALL)
    
    # 텍스트가 제목 패턴만 포함하는지 확인
    if title_pattern.match(cleaned_text):
        # 제목 패턴을 제거한 후 내용이 있는지 확인
        content_without_title = re.sub(r"(---|===).*?(---|===)", "", cleaned_text, flags=re.DOTALL).strip()
        return not content_without_title
    
    return False

def process_wikidocs_files(paths: List[str]) -> List[Document]:
    all_docs = []
    for path in paths:
        text = load_txt(path)
        docs = split_by_structure_with_code(text)
        all_docs.extend(docs)
    return all_docs

def extract_and_replace_code_blocks(text: str):
    code_blocks = []
    pattern = re.compile(r"<CODE_BLOCK>\s*```.*?\n.*?```\s*</CODE_BLOCK>", re.DOTALL)
    def replacer(match):
        code_blocks.append(match.group())
        return f"[[CODE:{len(code_blocks) - 1}]]"
    modified_text = pattern.sub(replacer, text)
    return modified_text, code_blocks

def split_protected_chunks(docs: List[Document], chunk_size=2000, chunk_overlap=50) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    chunked_docs = []

    for doc in docs:
        mod_text, code_blocks = extract_and_replace_code_blocks(doc.page_content)
        temp_doc = Document(page_content=mod_text, metadata=doc.metadata)
        split_docs = splitter.split_documents([temp_doc])

        for d in split_docs:
            # 실제로 사용된 [[CODE:X]]만 찾아서 복원
            used_codes = re.findall(r"\[\[CODE:(\d+)\]\]", d.page_content)
            for code_index in set(used_codes):
                code_index = int(code_index)
                if code_index < len(code_blocks):
                    d.page_content = d.page_content.replace(f"[[CODE:{code_index}]]", code_blocks[code_index])
            chunked_docs.append(d)

    return chunked_docs

In [53]:
# 1. Wikidocs 텍스트 파일들을 챕터 단위로 로딩 (설명 + 코드 보존)
docs = process_wikidocs_files([
    "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt",
    "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_02.txt",
    "C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_03.txt"
])

# 2. 임베딩 전용 chunk 단위로 분할 (코드블록 보호됨)
chunked_docs = split_protected_chunks(docs)

# 3. 확인
print(f"총 원본 챕터 수: {len(docs)}")
print(f"총 분할된 문서 수 (임베딩용): {len(chunked_docs)}")
print("--- 샘플 ---")
print(chunked_docs[1].page_content[:500])


총 원본 챕터 수: 184
총 분할된 문서 수 (임베딩용): 1241
--- 샘플 ---
--- CH01 LangChain 시작하기 ---


In [54]:
# Document 객체에서 텍스트 내용만 추출
text_contents = [doc.page_content for doc in chunked_docs]

test_db = Chroma.from_texts(
    texts=text_contents,
    embedding=embeddings,
    persist_directory=r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\test_db"
)

In [55]:
test_db.as_retriever(search_type="similarity", search_kwargs={"k": 5}).invoke('랭체인')

[Document(metadata={}, page_content='--- 04. FlashRank Reranker ---'),
 Document(metadata={}, page_content='--- 04. FlashRank Reranker ---'),
 Document(metadata={}, page_content='--- 04. LLM 체인 라우팅(RunnableLambda, RunnableBranch) ---'),
 Document(metadata={}, page_content='--- 04. LLM 체인 라우팅(RunnableLambda, RunnableBranch) ---'),
 Document(metadata={}, page_content='<CODE_BLOCK>\n```\nPassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked\n```\n</CODE_BLOCK>\nPassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked\n컬럼에 대한 값을 조회하는 예제입니다.\n# 열 작업 예시입니다.\ndf_query = "Age column 을 조회해 주세요."\n# 프롬프트 템플릿을 설정합니다.\nprompt = PromptTemplate(\ntemplate="Answer the user query.\\n{format_instructions}\\n{query}\\n",\ninput_variables=["query"],  # 입력 변수 설정\npartial_variables={\n"format_instructions": parser.get_format_instructions()\n},  # 부분 변수 설정\n)\n# 체인 생성\nchain = prompt | model | parser\n# 체인 실행\nparser_output = chain.invok

In [56]:
for doc in docs:
    print(f"문서 메타데이터:")
    for key, value in doc.metadata.items():
        if key == 'level':
            print(f"  {'  ' * (int(value) - 1)}└─ {key}: {value}")
        else:
            print(f"  {key}: {value}")
    print("-" * 50)

문서 메타데이터:
  chapter_info: None
  section_info: None
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: None
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 01. 설치 영상보고 따라하기
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 02. OpenAI API 키 발급 및 테스트
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 03. LangSmith 추적 설정
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 04. OpenAI API 사용(GPT-4o 멀티모달)
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 05. LangChain Expression Language(LCEL)
--------------------------------------------------
문서 메타데이터:
  chapter_info: CH01 LangChain 시작하기
  section_info: 06. LCEL 인터페이스
---

In [15]:
# import re
# from langchain_core.documents import Document

# def extract_code_blocks_from_documents(docs: List[Document]) -> List[str]:
#     """
#     각 Document 객체 내의 <CODE_BLOCK>...</CODE_BLOCK> 구간만 추출하여 리스트로 반환
#     """
#     code_blocks = []
#     pattern = re.compile(r"<CODE_BLOCK>\s*```.*?\n.*?```\s*</CODE_BLOCK>", re.DOTALL)

#     for doc in docs:
#         matches = pattern.findall(doc.page_content)
#         code_blocks.extend(matches)

#     return code_blocks

# chunked_docs = split_protected_chunks(docs)

# code_blocks = extract_code_blocks_from_documents(chunked_docs)

# # 출력 확인
# for i, code in enumerate(code_blocks[100:150]):
#     print(f"🔹 Code Block {i + 1}:\n{code}\n{'-'*60}")


In [57]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 로컬 LLM (gemma:3.12b)
# llm = Ollama(model="gemma3:12b")
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# 4. 메타데이터 필드 정의 (사용 필드만)
metadata_field_info = [
    {"name": "chapter_info", "type": "string"},
    {"name": "section_info", "type": "string"},
]

# 5. ParentDocumentRetriever 구성
retriever = test_db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# 6. QA Chain
prompt = PromptTemplate.from_template(
    '''
        당신은 랭체인과 관련된 질문에 대해 답변하는 전문가입니다.
        랭체인과 관련된 질문이 아니라면, "랭체인과 관련된 질문이 아닙니다"라고 답변해주세요.
        답변은 한국어로 답변해주세요.

        question : 
        {question}

        context :
        {context}

        answer : 
    '''
)

# LCEL 방식으로 체인 구성
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 7. 질문 실행
question = "LangGraph에 대해 설명해"
result = qa_chain.invoke(question)

# 8. 출력
print("🧠 답변:\n", result)

🧠 답변:
 LangGraph는 LangChain의 구성 요소 중 하나로, 주로 챗봇이나 에이전트를 구축하는 데 사용됩니다. LangGraph는 다양한 언어 모델과 데이터 소스를 연결하여 복잡한 대화 흐름을 관리하고, 사용자와의 상호작용을 보다 자연스럽고 효율적으로 만들어 줍니다. 이를 통해 개발자는 다양한 기능을 가진 챗봇을 쉽게 구축할 수 있으며, LangChain의 다른 도구들과 통합하여 더욱 강력한 애플리케이션을 개발할 수 있습니다. LangGraph는 특히 대화의 맥락을 이해하고, 적절한 응답을 생성하는 데 중점을 두고 설계되었습니다.


# ParentDocumentRetriever 구성

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(r'C:\Users\user\Documents\GitHub\Presentation-Agent\.env')

True

In [2]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [3]:
loaders = [
    # 파일을 로드합니다.
    TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_01.txt"),
    TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_02.txt"),
    TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_03.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

In [4]:
# # 자식 분할기를 생성합니다.
# child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# # 임베딩 모델 설정
# # embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
# embeddings = OpenAIEmbeddings()

# # DB를 생성합니다.
# vectorstore = Chroma(
#     collection_name="test", 
#     embedding_function=embeddings, 
#     # persist_directory = r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\test"
# )

# store = InMemoryStore()

# # Retriever 를 생성합니다.
# retriever = ParentDocumentRetriever(
#     vectorstore=vectorstore,
#     docstore=store,
#     child_splitter=child_splitter,
# )

In [5]:
# # 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
# retriever.add_documents(docs, ids=None, add_to_docstore=True)

# # 유사도 검색을 수행합니다.
# sub_docs = vectorstore.similarity_search("Langchain")

In [6]:
# sub_docs

[Document(id='1184519c-d2c9-462f-ba18-df1113dd490f', metadata={'doc_id': 'de864918-a334-4bae-bb3b-ce948fde3499', 'source': 'C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt'}, page_content='==================================================\n\n\n--- 03. LangChain Hub ---'),
 Document(id='ee3ed590-ed81-4822-b0b1-18136e76cced', metadata={'doc_id': 'de864918-a334-4bae-bb3b-ce948fde3499', 'source': 'C:\\Users\\user\\Documents\\GitHub\\Presentation-Agent\\data\\txt\\wikidocs_01.txt'}, page_content='Overall, the LangChain documentation provides a comprehensive guide to using the LangChain framework and LCEL for building and executing complex chains of operations involving language models and other components. It covers both basic and advanced use cases, offering practical examples and encouraging community involvement.The provided documents from LangChain cover a range of topics related to the LangChain Expression Language (LCEL) and its applications, includ

In [7]:
# print(sub_docs[0].page_content)



--- 03. LangChain Hub ---


In [8]:
# retrieved_docs = retriever.invoke("LangChain")

# # 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
# print(
#     f"문서의 길이: {len(retrieved_docs[0].page_content)}",
#     end="\n\n=====================\n\n",
# )

# # 문서의 일부를 출력합니다.
# print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 1800480


는 모듈식으로 설계되어, 사용하기 쉽습니다. 이는 개발자가 LangChain 프레임워크를 자유롭게 활용할 수 있게 해줍니다.
즉시 사용 가능한 체인 🚀
고수준 작업을 수행하기 위한 컴포넌트의 내장 조합을 제공합니다.
이러한 체인은 개발 과정을 간소화하고 속도를 높여줍니다.
주요 모듈 📌
모델 I/O 📃
프롬프트 관리, 최적화 및 LLM과의 일반적인 인터페이스와 작업을 위한 유틸리티를 포함합니다.
검색 📚
'데이터 강화 생성'에 초점을 맞춘 이 모듈은 생성 단계에서 필요한 데이터를 외부 데이터 소스에서 가져오는 작업을 담당합니다.
에이전트 🤖
언어 모델이 어떤 조치를 취할지 결정하고, 해당 조치를 실행하며, 관찰하고, 필요한 경우 반복하는 과정을 포함합니다.
LangChain을 활용하면, 언어 모델 기반 애플리케이션의 개발을 보다 쉽게 시작할 수 있으며, 필요에 맞게 기능을 맞춤 설정하고, 다양한 데이터 소스와 통합하여 복잡한 작업을 처리할 수 있게 됩니다.



In [4]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 250, separators=['==================================================', '---.*?---', '===.*?==='])
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 100)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_knowledge", 
    embedding_function=OpenAIEmbeddings(),
    persist_directory = r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\split_knowledge"
)
# 부모 문서의 저장 계층입니다.
parent_store = InMemoryStore()

In [5]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=parent_store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

In [6]:
retriever.add_documents(docs)



--- 03. LangChain Hub ---


In [7]:
# # 유사도 검색을 수행합니다.
# sub_docs = vectorstore.similarity_search("딥러닝")

# # sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
# print(sub_docs[0].page_content)

=== 딥 러닝을 이용한 자연어 처리 입문 ===


--- 딥 러닝을 이용한 자연어 처리 입문 ---


In [8]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("딥러닝")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(retrieved_docs[0].page_content)

=== 딥 러닝을 이용한 자연어 처리 입문 ===


--- 딥 러닝을 이용한 자연어 처리 입문 ---

25년 1월 기준: 누적 조회수: 1,700만 베스트셀러
많은 분들의 피드백으로 수 년간 보완된 현업 연구원들이 작성한 딥 러닝 자연어 처리 교재 입문서입니다.
Q) 입문자도 공부 가능한가요?
A) 이 책은 애초 AI를 아예 처음 공부하는 입문자가 타겟입니다. 파이썬을 어느 정도 할 줄 아신다면 AI 공부를 할 수 있습니다.
유료 E-book/전체 PDF 파일 (https://wikidocs.net/buy/ebook/2155)
A) 거의 90% 이상의 내용을 현재 무료로 공개했습니다. 그러니 무료로 편하게 입문하시기 바랍니다.
단, 파인 튜닝 등 일부심화 내용은 유료 E-book/전체 PDF 파일에서만 볼 수 있습니다.
온라인 강의 (https://bit.ly/4fWkdRa)온라인 강의는 없나요?
A) LLM 파인 튜닝을 다루는 입문용 온라인 강의가 존재합니다.
🎉할인 쿠폰 코드 입력: '파인튜닝' (할인율 : 20%)
오프라인 강의 (https://learningspoons.com/course/detail/llm-master/)오프라인 강의는 없나요?
A) 강남에서 매주 현장에서 질답을 받으며 진행되는 LLM 파인 튜닝을 다루는 입문용 오프라인 강의가 존재합니다.
존재 (https://wikidocs.net/book/2788)파이토치 버전은 없나요?
A) 존재합니다. 목차는 거의 동일하므로 선호하는 책으로 구매하세요.
Q) 입문자들이 쉽게 질문하고 소통할 수 있는 공간은 없나요?
A) 자연어 처리 입문자들을 위한 오픈 카톡방: https://open.kakao.com/o/gciNJmPg
댓글 또는 피드백(질문/지적) 또는 이메일 환영합니다.
각 내용에 대한 페이지마다 댓글 버튼 옆을 보면 피드백 버튼이 있습니다.
위키독스 회원가입이 번거로우시다면 피드백 버튼으로 의견주셔도 댓글로 답변드립니다.
감사합니다.


In [ ]:
for doc in retrieved_docs[:5] :
    print(doc.page_content)

In [16]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI
# # from langchain.llms import Ollama  # 로컬 LLM 쓸 경우 사용

# # LLM 초기화
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# # llm = Ollama(model="gemma3:12b")  # 로컬 모델 쓸 경우

# # 시스템 프롬프트 불러오기
# with open("C:/Users/user/Documents/GitHub/Presentation-Agent/data/txt/DeePrint.txt", encoding="utf-8") as f:
#     pt_context = f.read()

# # 템플릿 정의
# prompt = ChatPromptTemplate.from_messages([
#     ("system", f"당신은 발표자료에 대한 내용을 질문받으면 그에 대한 답을 하는 AI 에이전트입니다.\n다음은 발표자료에 대한 배경 정보입니다:\n\n{pt_context}\n 답변은 간결하게 100토큰 이내로 답변해주세요."),
#     ("human", "질문: {question}\n\n문서:\n{documents}")
# ])

# # LCEL 체인 구성
# rag_chain = (
#     {
#         "question": RunnablePassthrough(),
#         "documents": retriever
#     }
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# # 질문 목록
# questions = [
#     "이 프로젝트의 시스템 구조는 어떻게 되나요?",
#     "프로젝트를 좀 더 고도화한다면 어떤 방안을 생각해보셨나요?",
#     "프로젝트의 향후 발전 방향으로 Langchain을 활용하여 RAG 시스템을 고도화하는게 좋아보이는데 거기까지 고려해보셨을까요?"
# ]

# # 질문에 대한 응답 생성
# for q in questions:
#     try:
#         response = rag_chain.invoke(q)
#         print(f"\n❓ 질문: {q}\n💬 답변: {response}\n" + "-"*50)
#     except Exception as e:
#         print(f"\n❌ 오류 발생: {e}\n" + "-"*50)



❓ 질문: 이 프로젝트의 시스템 구조는 어떻게 되나요?
💬 답변: 이 프로젝트의 시스템 구조는 크게 데이터 수집, 모델 학습 및 평가, 성능 개선, 그리고 서비스 구현으로 나눌 수 있습니다. 데이터 수집 단계에서는 아동 그림 데이터를 YOLO 모델로 정리하고, 모델 학습 단계에서는 YOLO11n 모델을 통해 그림 요소를 탐지합니다. 이후 성능 개선을 위해 이미지 해상도를 조정하고 학습 속도를 최적화합니다. 마지막으로, 학습된 모델을 서비스에 적용하여 HTP 검사 결과를 자동으로 분석하고 해석하는 기능을 구현합니다.
--------------------------------------------------

❓ 질문: 프로젝트를 좀 더 고도화한다면 어떤 방안을 생각해보셨나요?
💬 답변: 프로젝트를 고도화하기 위해, 다음과 같은 방안을 고려할 수 있습니다. 첫째, 다양한 아동 미술 데이터를 추가로 수집하여 모델의 일반화 능력을 향상시킬 수 있습니다. 둘째, 심리 진단의 정확성을 높이기 위해 다중 모달 학습을 도입하여 텍스트와 이미지 데이터를 통합 분석할 수 있습니다. 셋째, 사용자 피드백을 반영한 지속적인 모델 업데이트 및 성능 모니터링 시스템을 구축하여 실시간으로 개선할 수 있습니다.
--------------------------------------------------

❓ 질문: 프로젝트의 향후 발전 방향으로 Langchain을 활용하여 RAG 시스템을 고도화하는게 좋아보이는데 거기까지 고려해보셨을까요?
💬 답변: 네, Langchain을 활용하여 RAG 시스템을 고도화하는 방향은 매우 유망합니다. 이를 통해 데이터 처리 및 응답 생성의 효율성을 높일 수 있습니다. 현재 시스템의 한계를 보완하고, 더 나은 사용자 경험을 제공하기 위해 Langchain의 도입을 고려할 수 있습니다.
--------------------------------------------------


In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI

# ✅ LLM 설정 (ChatOpenAI 또는 Ollama 사용 가능)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# from langchain.llms import Ollama
# llm = Ollama(model="gemma3:12b")

# ✅ 시스템 프롬프트 로드
with open("C:/Users/user/Documents/GitHub/Presentation-Agent/data/txt/pt_context.txt", encoding="utf-8") as f:
    pt_context = f.read()

# ✅ 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", f"당신은 발표자료에 대한 내용을 질문받으면 그에 대한 답을 하는 AI 에이전트입니다.\n"
               f"다음은 발표자료에 대한 배경 정보입니다:\n\n{pt_context}\n\n"
               f"답변은 간결하게 100토큰 이내로 작성해주세요.\n"
               f"사용자가 처음 물어봐서 이전 대화 내용이 없어도 질문에 대한 대답을 하세요. \n"
               f"사용자가 이전 대화 내용에 대해 물어보면, 대화 기록을 확인하여 정확히 답변해주세요."), 
    ("human", "{question}"),
    ("human", "문서:\n{documents}"),
    ("human", "이전 대화 내용:\n{chat_history}")
])

# ✅ 대화 이력 저장용 함수 및 저장소
chat_histories = {}

def get_message_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

# ✅ 이전 대화 내용을 문자열로 변환하는 함수
def format_chat_history(chat_history):
    formatted_history = []
    for message in chat_history:
        if hasattr(message, 'content') and hasattr(message, 'type'):
            formatted_history.append(f"{message.type}: {message.content}")
    return "\n".join(formatted_history)

# ✅ RAG LCEL 체인 구성
rag_chain = (
    {
        "question": lambda x: x["question"],
        "documents": lambda x: retriever.invoke(x["question"]),
        "chat_history": lambda x: format_chat_history(x.get("chat_history", []))
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ RAG + 메모리 연결된 체인 구성
chat_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history=get_message_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

# ✅ 질의 테스트
questions = [
    "이 프로젝트의 시스템 구조는 어떻게 되나요?",
    "프로젝트를 좀 더 고도화한다면 어떤 방안을 생각해보셨나요?",
    "LangChain 기반으로 RAG 시스템을 고도화하는 것도 고려했나요?",
    "제가 처음으로 물어본 질문이 뭐였나요?"
]

session_id = "user-session-1"
for i, q in enumerate(questions):
    try:
        response = chat_chain.invoke(
            {"question": q},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"\n❓ 질문: {q}\n💬 답변: {response}\n" + "-" * 50)
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}\n" + "-" * 50)


❓ 질문: 이 프로젝트의 시스템 구조는 어떻게 되나요?
💬 답변: 프로젝트의 시스템 구조는 사용자가 발표 자료를 업로드하면, AI가 내용을 분석하여 발표 대본을 생성하고, 음성으로 발표까지 진행하는 방식입니다. VectorDB에 문서를 저장하고, LLM이 이를 분석하여 발표 내용을 구성합니다. FastAPI를 통해 API 서비스를 제공하고, Streamlit을 활용하여 사용자 인터페이스를 구축합니다.
--------------------------------------------------

❓ 질문: 프로젝트를 좀 더 고도화한다면 어떤 방안을 생각해보셨나요?
💬 답변: 프로젝트를 고도화하기 위한 방안으로는 실시간 상호작용 기능 추가, 사용자 맞춤 발표 스타일 적용, 디지털 아바타 활용, 그리고 다양한 도메인으로의 확장이 있습니다. 이를 통해 발표 경험을 더욱 자연스럽고 개인화된 방식으로 개선할 수 있습니다.
--------------------------------------------------

❓ 질문: LangChain 기반으로 RAG 시스템을 고도화하는 것도 고려했나요?
💬 답변: LangChain 기반으로 RAG 시스템을 고도화하는 방안은 고려하지 않았습니다. 현재 프로젝트에서는 RAG와 에이전트를 독립적으로 구현하여 운용하고 있으며, LangChain의 사용은 필수가 아닙니다. 대신, LLM의 동작 원리와 RAG의 원리를 이해하는 것이 더 중요하다고 판단하고 있습니다.
--------------------------------------------------

❓ 질문: 제가 처음으로 물어본 질문이 뭐였나요?
💬 답변: 이전 대화에서 사용자는 프로젝트의 시스템 구조와 고도화 방안에 대해 질문하였고, AI는 시스템 구조를 설명하고 고도화 방안으로 실시간 상호작용, 맞춤 발표 스타일, 디지털 아바타 활용 등을 언급했습니다. 또한, LangChain 기반 RAG 시스템 고도화에 대한 질문에 AI는 현재 프로젝트에서 LangChain 사용이 필수가 아

In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API 키 정보 로드
load_dotenv(r'C:\Users\user\Documents\GitHub\Presentation-Agent\.env')

from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

# loaders = [
#     # 파일을 로드합니다.
#     TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_01.txt"),
#     TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_02.txt"),
#     TextLoader(r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\txt\wikidocs_03.txt"),
# ]

# docs = []
# for loader in loaders:
#     # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
#     docs.extend(loader.load())

# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 250, separators=['==================================================', '---.*?---', '===.*?==='])
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 100)

# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_knowledge", 
    embedding_function=OpenAIEmbeddings(),
    persist_directory = r"C:\Users\user\Documents\GitHub\Presentation-Agent\data\db\chromadb\split_knowledge"
)

# retriever.add_documents(docs)

# 부모 문서의 저장 계층입니다.
parent_store = InMemoryStore()

retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=parent_store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI

# ✅ LLM 설정 (ChatOpenAI 또는 Ollama 사용 가능)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# from langchain.llms import Ollama
# llm = Ollama(model="gemma3:12b")

# ✅ 시스템 프롬프트 로드
with open("C:/Users/user/Documents/GitHub/Presentation-Agent/data/txt/pt_context.txt", encoding="utf-8") as f:
    pt_context = f.read()

# ✅ 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", f"당신은 발표자료에 대한 내용을 질문받으면 그에 대한 답을 하는 AI 에이전트입니다.\n"
               f"다음은 발표자료에 대한 배경 정보입니다:\n\n{pt_context}\n\n"
               f"답변은 간결하게 100토큰 이내로 작성해주세요.\n"
               f"사용자가 처음 물어봐서 이전 대화 내용이 없어도 질문에 대한 대답을 하세요. \n"
               f"사용자가 이전 대화 내용에 대해 물어보면, 대화 기록을 확인하여 정확히 답변해주세요."), 
    ("human", "{question}"),
    ("human", "문서:\n{documents}"),
    ("human", "이전 대화 내용:\n{chat_history}")
])

# ✅ 대화 이력 저장용 함수 및 저장소
chat_histories = {}

def get_message_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

# ✅ 이전 대화 내용을 문자열로 변환하는 함수
def format_chat_history(chat_history):
    formatted_history = []
    for message in chat_history:
        if hasattr(message, 'content') and hasattr(message, 'type'):
            formatted_history.append(f"{message.type}: {message.content}")
    return "\n".join(formatted_history)

# ✅ RAG LCEL 체인 구성
rag_chain = (
    {
        "question": lambda x: x["question"],
        "documents": lambda x: retriever.invoke(x["question"]),
        "chat_history": lambda x: format_chat_history(x.get("chat_history", []))
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ RAG + 메모리 연결된 체인 구성
chat_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history=get_message_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

# ✅ 질의 테스트
questions = [
    "이 프로젝트의 시스템 구조는 어떻게 되나요?",
    "프로젝트를 좀 더 고도화한다면 어떤 방안을 생각해보셨나요?",
    "LangChain 기반으로 RAG 시스템을 고도화하는 것도 고려했나요?",
    "제가 이 대화에서 처음으로 물어본 질문이 뭐였나요?"
]

session_id = "user-session-1"
for i, q in enumerate(questions):
    try:
        response = chat_chain.invoke(
            {"question": q},
            config={"configurable": {"session_id": session_id}}
        )
        print(f"\n❓ 질문: {q}\n💬 답변: {response}\n" + "-" * 50)
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}\n" + "-" * 50)


❓ 질문: 이 프로젝트의 시스템 구조는 어떻게 되나요?
💬 답변: 프로젝트의 시스템 구조는 사용자가 발표 자료를 업로드하면, AI가 내용을 분석하여 발표 대본을 생성하고 음성으로 발표까지 진행하는 방식입니다. 문서는 VectorDB에 저장되고, LLM이 이를 분석하여 발표 내용을 구성합니다. FastAPI를 통해 API 서비스를 제공하고, Streamlit을 활용하여 사용자 인터페이스(UI)를 구축합니다.
--------------------------------------------------

❓ 질문: 프로젝트를 좀 더 고도화한다면 어떤 방안을 생각해보셨나요?
💬 답변: 프로젝트를 고도화하기 위한 방안으로는 실시간 상호작용 기능 추가, 인간처럼 자연스럽게 말하는 음성 합성, 사용자별 맞춤 발표 스타일 적용, 디지털 아바타를 활용한 발표 기능, 그리고 도메인 확장이 있습니다. 이러한 개선을 통해 더욱 자연스러운 발표 경험을 제공하고 다양한 환경에서 활용할 수 있도록 할 계획입니다.
--------------------------------------------------

❓ 질문: LangChain 기반으로 RAG 시스템을 고도화하는 것도 고려했나요?
💬 답변: 네, LangChain 기반으로 RAG 시스템을 고도화하는 것도 고려하고 있습니다. 이를 통해 문서 검색 및 분석의 효율성을 높이고, 사용자에게 더욱 정확하고 관련성 높은 발표 대본을 제공할 수 있을 것입니다. RAG 시스템의 개선은 발표의 품질을 향상시키는 데 중요한 역할을 할 것입니다.
--------------------------------------------------

❓ 질문: 제가 이 대화에서 처음으로 물어본 질문이 뭐였나요?
💬 답변: 이전 대화 내용에서 사용자가 처음으로 물어본 질문은 "이 프로젝트의 시스템 구조는 어떻게 되나요?"입니다.
--------------------------------------------------


In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain
# from langchain_openai import ChatOpenAI
# from langchain.llms import Ollama

# prompt = ChatPromptTemplate.from_template("""
# 다음 문서를 기반으로 질문에 답변해주세요.
# 문서에 관련 정보가 없다면, "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 답변하세요.

# 문서:
# {context}

# 질문:
# {input}
# """)

# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=1000)
# # llm = Ollama(model="gemma3:12b", temperature = 0)

# # document_variable_name을 제거해야 오류 없음
# document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# chain = create_retrieval_chain(retriever, document_chain)

# response = chain.invoke({"input": "이 프로젝트를 듣고 난 후에 생길 수 있는 예상 질문을 10개 만들고 그 질문에 대한 답변을 만들어"})
# print(response["answer"])